## Go

Instalar Go

In [ ]:
!apt install golang-go

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  golang-1.18-go golang-1.18-src golang-src
Suggested packages:
  bzr | brz mercurial subversion
The following NEW packages will be installed:
  golang-1.18-go golang-1.18-src golang-go golang-src
0 upgraded, 4 newly installed, 0 to remove and 45 not upgraded.
Need to get 82.3 MB of archives.
After this operation, 436 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 golang-1.18-src all 1.18.1-1ubuntu1.1 [16.2 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 golang-1.18-go amd64 1.18.1-1ubuntu1.1 [66.0 MB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 golang-src all 2:1.18~0ubuntu2 [4,438 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 golang-go amd64 2:1.18~0ubuntu2 [41.8 kB]
Fetched 82.3 MB in 6s (14.6 MB/s)
Selecting previously unselected pa

In [ ]:
%%writefile concurrencia.go

package main

import (
	"errors"
	"fmt"
	"os"
	"strings"
	"sync"
)

// struct containing the data to obtain
type information struct {
	notEmptyLines      int
	words              int
	characters         int
	spaces             int
	frequencies        map[string]int
	mostFrequentWord   string
	mostFrequentAmount int
}

// string method to print information on screen
func (i information) String() string {
	return fmt.Sprintf("Not empty lines: %d\nWord amount: %d\nCharacter amount: %d\nSpaces amount: %d\nMost frequent word: \"%s\" with count = %d", i.notEmptyLines,
		i.words, i.characters, i.spaces, i.mostFrequentWord, i.mostFrequentAmount)
}

// global variables
var (
	info  information // the structure in which the threads will write their information
	mutex sync.Mutex  // semaphore to synchronize access to the info global variable
)

func main() {
	if len(os.Args) != 2 {
		fmt.Println("FilePath argument is required")
		return
	}

	filePath := os.Args[1]

	bytes, err := os.ReadFile(filePath)
	if err != nil {
		if errors.Is(err, os.ErrNotExist) {
			fmt.Println("The file ", filePath, " does not exist")
			return
		}

		panic(err)
	}

	text := string(bytes)
	info.frequencies = make(map[string]int)
	var wg sync.WaitGroup

	lines := strings.Split(text, "\n")

	for _, line := range lines {
		line = strings.TrimSpace(line) //remove \r at the end of the line
		if line != "" {
			wg.Add(1)
			go processLine(line, &wg)
		}
	}

	wg.Wait()

	info.mostFrequentWord, info.mostFrequentAmount = mostFrequent()

	fmt.Println(info)
}

func processLine(line string, wg *sync.WaitGroup) {
	words := countWords(line)
	chars := countChars(line)
	spaces := countSpaces(line)
	freq := calculateFrequency(line)
	mutex.Lock()
	info.notEmptyLines++
	info.words += words
	info.characters += chars
	info.spaces += spaces
	for word, count := range freq {
		info.frequencies[word] += count
	}
	mutex.Unlock()
	wg.Done()
}

func countWords(line string) int {
	return len(strings.Split(line, " "))
}

func countChars(line string) int {
	return len(line)
}

func countSpaces(line string) int {
	return strings.Count(line, " ")
}

func calculateFrequency(line string) map[string]int {
	wordFreq := make(map[string]int)
	for _, word := range strings.Split(line, " ") {
		wordFreq[word]++
	}
	return wordFreq
}

func mostFrequent() (mostFreqWord string, max int) {
	max = 0
	mostFreqWord = ""
	for word, count := range info.frequencies {
		if count > max {
			max = count
			mostFreqWord = word
		}
	}
	return
}



Writing concurrencia.go


In [ ]:
!go run concurrencia.go texto

/bin/bash: line 1: go: command not found


## C++

In [ ]:
%%writefile concurrencia.cpp
#include <iostream>
#include <fstream>
#include <string>
#include <vector>
#include <unordered_map>
#include <mutex>
#include <thread>
#include <sstream>
#include <algorithm>

using namespace std;

struct Informacion
{
    int lineasNoVacias = 0;
    int palabras = 0;
    int caracteres = 0;
    int espacios = 0;
    unordered_map<string, int> frecuencias;
    string palabraMasFrecuente;
    int cantidadMasFrecuente = 0;
};

mutex mtx;
Informacion info;

int ContarEspacios(const string& linea)
{
    return count(linea.begin(), linea.end(), ' ');
}

void ProcesarLinea(const string& linea)
{
    int palabras = 0;
    int caracteres = 0;
    int espacios = 0;
    unordered_map<string, int> frecuenciaPalabras;

    string palabra;
    istringstream iss(linea);

    while (iss >> palabra)
    {
        frecuenciaPalabras[palabra]++;
        palabras++;
        caracteres += palabra.length();
    }

    espacios = ContarEspacios(linea);

    lock_guard<mutex> guard(mtx);
    info.lineasNoVacias++;
    info.palabras += palabras;
    info.caracteres += caracteres;
    info.espacios += espacios;
    for (const auto& palabra : frecuenciaPalabras)
    {
        info.frecuencias[palabra.first] += palabra.second;
    }
}

int main(int argc, char* argv[])
{
    if (argc != 2)
    {
        cerr << "Se requiere el argumento del archivo" << endl;
        return 1;
    }

    ifstream archivo(argv[1]);
    if (!archivo)
    {
        cerr << "El archivo " << argv[1] << " no existe" << endl;
        return 1;
    }

    vector<thread> hilos;

    string linea;
    while (getline(archivo, linea))
    {
        if (!linea.empty())
        {
            hilos.emplace_back(ProcesarLinea, linea);
        }
    }

    for (auto& hilo : hilos)
    {
        hilo.join();
    }

    for (const auto& palabra : info.frecuencias)
    {
        if (palabra.second > info.cantidadMasFrecuente)
        {
            info.cantidadMasFrecuente = palabra.second;
            info.palabraMasFrecuente = palabra.first;
        }
    }

    cout << "Cantidad de líneas no vacías: " << info.lineasNoVacias << endl;
    cout << "Cantidad de palabras: " << info.palabras << endl;
    cout << "Cantidad de caracteres: " << info.caracteres << endl;
    cout << "Cantidad de espacios: " << info.espacios << endl;
    cout << "Palabra más frecuente: \"" << info.palabraMasFrecuente << "\" con cantidad = " << info.cantidadMasFrecuente << endl;

    return 0;
}



Overwriting concurrencia.cpp


In [ ]:
%%writefile texto_c.txt
Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam,
quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit
in voluptate velit esse cillum dolore eu fugiat nulla pariatur.
Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

Overwriting texto_c.txt


In [ ]:
!g++ -o concurrencia.bin concurrencia.cpp

In [ ]:
!./concurrencia.bin "texto_c.txt"

Cantidad de líneas no vacías: 4
Cantidad de palabras: 69
Cantidad de caracteres: 377
Cantidad de espacios: 66
Palabra más frecuente: "in" con cantidad = 4


## Java

In [ ]:
%%writefile hilos.java

package paquete;
import java.util.ArrayList;
import java.util.Scanner;
import java.util.concurrent.Semaphore;

public class CruceGranjero3 {

    private static final Semaphore semaphore1 = new Semaphore(1); // Inicialmente solo un hilo puede adquirir el semáforo

    private static ArrayList<String> arrayListA = new ArrayList<>();
    private static ArrayList<String> arrayListB = new ArrayList<>();

    // Métodos para simular el cruce de los elementos
    private static  void cruzar(String pasajero) {
        System.out.println("Cruzando de punto A hacia B...");
        System.out.println("El " + pasajero + " se encuentran en el punto B.");

        arrayListB.add("granjero");
        arrayListA.remove("granjero");

        if (!pasajero.equals("granjero")) {
        	arrayListB.add(pasajero);
            arrayListA.remove(pasajero);
		}
    }


    private static  void regresar(String pasajero) {
        System.out.println("Regresando de punto B hacia A...");
        System.out.println("El " + pasajero + " ha cruzado de regreso al punto A.");

        arrayListA.add(pasajero);
        arrayListB.remove(pasajero); // Eliminar el elemento "zorro" del ArrayList

        if (!pasajero.equals("granjero")) {

            arrayListA.add("granjero");
            arrayListB.remove("granjero");
		}
    }


    // Método principal
    public static void main(String[] args) {

        Thread hilo1 = new Thread(() -> {
        		try {
					semaphore1.acquire();
					cruce();
				} catch (InterruptedException e) {
					// TODO Auto-generated catch block
					e.printStackTrace();
				}finally {
					semaphore1.release();
				}

        });

        Thread hilo2 = new Thread(() -> {
        	try {
				semaphore1.acquire();
				cruce();
			} catch (InterruptedException e) {
				// TODO Auto-generated catch block
				e.printStackTrace();
			}finally {
				semaphore1.release();
			}


        });

        hilo1.start();
        hilo2.start();


        try {
            hilo1.join();
            hilo2.join();
            System.out.println("Hilos finalizado OK");
        } catch (InterruptedException e) {
            e.printStackTrace();
        }

        System.out.println("Fin Exitoso!");

    }
	private static void cruce() {


    	arrayListA.add("granjero");
        arrayListA.add("zorro");
        arrayListA.add("pollo");
        arrayListA.add("maiz");

        Scanner scanner = new Scanner(System.in);

        System.out.println("Se encuentran en el punto A: " + arrayListA);
    	System.out.println("Se encuentran en el punto B: " + arrayListB);

    	boolean cruzado = true;
        while (true) {

        	String opcion;
			// Solicitar al usuario que ingrese una opción válida
        	if(cruzado) {
        		System.out.println("Con que pasajero desea cruzar: ");
        	}else {
        		System.out.println("Con que pasajero desea regresar: ");
        	}

        	 	do {
			            //System.out.print("¿Cruzar/Regresar con? (Zorro, Pollo, Maiz, Salir): ");
			            opcion = scanner.nextLine().toLowerCase();

			            if (opcion.equals("zorro") || opcion.equals("pollo") || opcion.equals("maiz") || opcion.equals("salir") || opcion.equals("granjero")) {
			            	if (cruzado) {
								if (!opcion.equals("salir") && !arrayListA.contains(opcion)) {
									System.out.println("No se encuentra en el punto A!. Por favor, ingresa una opción válida");
								}else {
									break;
								}
								//break;
							}else {
								if (!opcion.equals("salir") && !arrayListB.contains(opcion)) {
									System.out.println("No se encuentra en el punto B!. Por favor, ingresa una opción válida");
								}else {
									break;
								}
							}

			            } else {
			                System.out.println("Opción inválida. Por favor, ingresa una opción válida (Zorro, Pollo, Maiz, Salir): ");
			            }
        	 		} while (true);

            //String opcion = scanner.nextLine().toLowerCase();

            if (opcion.equals("salir")) {
                System.out.println("Fin del programa.");
                break;
            }

            try {

                // Realizar el cruce según la opción ingresada por el usuario
                if(cruzado) {
            		cruzar(opcion);
            		cruzado = false;
            	}else {
            		regresar(opcion);
            		cruzado = true;
            	}
            } catch (Exception e) {
                e.printStackTrace();
            }

            validarPasajeros();

            System.out.println("Se encuentran en el punto A: " + arrayListA);
        	System.out.println("Se encuentran en el punto B: " + arrayListB);
        	}


        	scanner.close();


	}


	private static void validarPasajeros() {

			if (arrayListA.contains("zorro") && arrayListA.contains("pollo") && !arrayListA.contains("granjero")){
				 System.out.println("El zorro se come al pollo en el punto A!. \nSe finaliza el programa.");
				 System.exit(0); // Terminar el programa

			}else if (arrayListA.contains("pollo") && arrayListA.contains("maiz")&& !arrayListA.contains("granjero")) {
				 System.out.println("El pollo se come el maiz en el punto A!. \nSe finaliza el programa.");
		         System.exit(0); // Terminar el programa
		    }

			if (arrayListB.contains("zorro") && arrayListB.contains("pollo") && !arrayListB.contains("granjero")){
				 System.out.println("El zorro se come al pollo en el punto B!. \nSe finaliza el programa.");
				 System.exit(0); // Terminar el programa
			}else if((arrayListB.contains("pollo") && arrayListB.contains("maiz") && !arrayListB.contains("granjero"))) {
				 System.out.println("El pollo se come el maiz en el punto B!. \nSe finaliza el programa.");
		         System.exit(0); // Terminar el programa
		    }

			 // Verificar si arrayList1 está vacío
		    if (arrayListA.isEmpty()) {
		        System.out.println("Todos estan en el punto B. Felicitaciones!");
		        System.exit(0); // Terminar el programa
	    	}


}
}
/*
 *  1-Llevar al pollo al punto B.
	2-Regresar solo al punto A.
	3-Llevar al zorro al punto B.
	4-Regresar con el pollo al punto A.
	5-Dejar al pollo en el punto A y llevar al maíz al punto B.
	6-Regresar solo al punto A.
 *	7-Llevar al pollo al punto B.
*/


Writing hilos.java


In [ ]:
! java hilos.java

Se encuentran en el punto A: [granjero, zorro, pollo, maiz]
Se encuentran en el punto B: []
Con que pasajero desea cruzar: 
pollo
Cruzando de punto A hacia B...
El pollo se encuentran en el punto B.
Se encuentran en el punto A: [zorro, maiz]
Se encuentran en el punto B: [granjero, pollo]
Con que pasajero desea regresar: 
granjero
Regresando de punto B hacia A...
El granjero ha cruzado de regreso al punto A.
Se encuentran en el punto A: [zorro, maiz, granjero]
Se encuentran en el punto B: [pollo]
Con que pasajero desea cruzar: 
zorro
Cruzando de punto A hacia B...
El zorro se encuentran en el punto B.
Se encuentran en el punto A: [maiz]
Se encuentran en el punto B: [pollo, granjero, zorro]
Con que pasajero desea regresar: 
pollo
Regresando de punto B hacia A...
El pollo ha cruzado de regreso al punto A.
Se encuentran en el punto A: [maiz, pollo, granjero]
Se encuentran en el punto B: [zorro]
Con que pasajero desea cruzar: 
maiz
Cruzando de punto A hacia B...
El maiz se encuentran en el 

## Python

In [ ]:
%%writefile ventas.txt
SUCURSAL,FECHA,PRODUCTO,MONTO
1,01-01-2024,pan,500
1,01-01-2024,leche,1700
1,01-01-2024,manteca,600
1,01-01-2024,manteca,600
1,01-01-2024,leche,1700
2,01-01-2024,pan,500
2,02-02-2024,oreos,800
1,02-02-2024,fernet,3000
2,02-02-2024,manteca,600
1,03-02-2024,pan,600
1,03-02-2024,fernet,3500
1,03-02-2024,crema,1500
2,10-05-2024,pan rallado,1900
2,10-05-2024,aceite,3000
1,10-10-2024,blue label,85500
1,10-11-2024,papel cocina,2000
1,10-11-2024,yogurt,450
2,10-11-2024,sal,1900
2,10-11-2024,pepitos,900
2,10-11-2024,aceite,2900
1,01-12-2024,papel cocina,1500
2,12-12-2024,fanta,3500
2,29-12-2024,coca,1500
2,29-12-2024,fanta,1200
2,29-12-2024,sidra,3000
2,30-12-2024,coca,1500
2,30-12-2024,fanta,1200
2,30-12-2024,sidra,3000
2,31-12-2024,coca,1500
2,31-12-2024,fanta,1200
2,31-12-2024,sidra,3000
2,01-01-2025,coca,1500
2,01-01-2025,fanta,1200
2,01-01-2025,sidra,3000


Writing ventas.txt


In [ ]:
%%writefile concurrencia.py
from multiprocessing import Pipe
import multiprocessing
import sys

SUCURSAL_IDX = 0
FECHA_IDX = 1
PRODUCTO_IDX = 2
MONTO_IDX = 3

def get_mas_ocurrencias(datos,tipo,unidad):
    mayores_ocurrencias = []
    mayor = 0
    for key,value in datos.items():
        if(value > mayor):
            mayor = value
            del mayores_ocurrencias [:]
            mayores_ocurrencias.append([key,value])
        elif(value == mayor):
            mayores_ocurrencias.append([key,value])


    if(len(mayores_ocurrencias) >= 3):
            counter = 0
            for data in mayores_ocurrencias:
                print(f"{tipo}: {data[0]} | {data[1]} {unidad}")
                counter += 1
                if(counter == 3):
                    break
    else:
        for data in mayores_ocurrencias:
            print(f"{tipo}: {data[0]} | {data[1]} {unidad}")

def procesar_ventas(archivo_ventas):
    with open(archivo_ventas) as arch:
        next(arch)
        registros_sucursal1 = []
        registros_sucursal2 = []
        for registro in arch:
            registro = registro.split(",")
            if(int(registro[SUCURSAL_IDX]) == 1):
                registros_sucursal1.append(registro)
            else:
                registros_sucursal2.append(registro)

    return registros_sucursal1,registros_sucursal2

def mostrar_resultados(num_sucursal,monto_total,producto_mas_vendido,fecha_mas_ventas,fechas_mas_monto_vendido):

    print(f"\nSucursal N°{num_sucursal}")
    print(f"El monto total de ventas de la sucursal {num_sucursal} es: ${monto_total}")

    print("------ Producto/s mas vendido/s: ------")
    get_mas_ocurrencias(producto_mas_vendido,"Producto", "unidades")

    print("------ Fechas con más ventas ------")
    get_mas_ocurrencias(fecha_mas_ventas,"Fecha","ventas")

    print("------ Fechas con mayor importe de ventas ------")
    get_mas_ocurrencias(fechas_mas_monto_vendido,"Fecha","pesos")

def sucursal1_handler(conn_recv, conn_env):
    num_sucursal = 1
    monto_total = 0
    producto_mas_vendido = {}
    fecha_mas_ventas = {}
    fechas_mas_monto_vendido = {}

    conn_env.close()
    ventas = conn_recv.recv()
    conn_recv.close()

    for venta in ventas:
        monto_total += int(venta[MONTO_IDX])

        if(venta[PRODUCTO_IDX] in producto_mas_vendido):
            producto_mas_vendido[venta[PRODUCTO_IDX]] += 1
        else:
            producto_mas_vendido[venta[PRODUCTO_IDX]] = 1

        if(venta[FECHA_IDX] in fecha_mas_ventas):
            fecha_mas_ventas[venta[FECHA_IDX]] += 1
        else:
            fecha_mas_ventas[venta[FECHA_IDX]] = 1

        if(venta[FECHA_IDX] in fechas_mas_monto_vendido):
            fechas_mas_monto_vendido[venta[FECHA_IDX]] += float(venta[MONTO_IDX])
        else:
            fechas_mas_monto_vendido[venta[FECHA_IDX]] = float(venta[MONTO_IDX])

    print(f"\nSucursal N°{num_sucursal}")
    print(f"El monto total de ventas de la sucursal {num_sucursal} es: ${monto_total}")

    print("------ Producto/s mas vendido/s: ------")
    get_mas_ocurrencias(producto_mas_vendido,"Producto", "unidades")

    print("------ Fechas con más ventas ------")
    get_mas_ocurrencias(fecha_mas_ventas,"Fecha","ventas")

    print("------ Fechas con mayor importe de ventas ------")
    get_mas_ocurrencias(fechas_mas_monto_vendido,"Fecha","pesos")

    sys.exit(0)

def sucursal2_handler(conn_recv, conn_env):
    num_sucursal = 2
    monto_total = 0
    producto_mas_vendido = {}
    fecha_mas_ventas = {}
    fechas_mas_monto_vendido = {}

    conn_env.close()
    ventas = conn_recv.recv()
    conn_recv.close()

    for venta in ventas:
        monto_total += int(venta[MONTO_IDX])

        if(venta[PRODUCTO_IDX] in producto_mas_vendido):
            producto_mas_vendido[venta[PRODUCTO_IDX]] += 1
        else:
            producto_mas_vendido[venta[PRODUCTO_IDX]] = 1

        if(venta[FECHA_IDX] in fecha_mas_ventas):
            fecha_mas_ventas[venta[FECHA_IDX]] += 1
        else:
            fecha_mas_ventas[venta[FECHA_IDX]] = 1

        if(venta[FECHA_IDX] in fechas_mas_monto_vendido):
            fechas_mas_monto_vendido[venta[FECHA_IDX]] += int(venta[MONTO_IDX])
        else:
            fechas_mas_monto_vendido[venta[FECHA_IDX]] = int(venta[MONTO_IDX])

    print(f"\nSucursal N°{num_sucursal}")
    print(f"El monto total de ventas de la sucursal {num_sucursal} es: ${monto_total}")

    print("------ Producto/s mas vendido/s: ------")
    get_mas_ocurrencias(producto_mas_vendido,"Producto", "unidades")

    print("------ Fechas con más ventas ------")
    get_mas_ocurrencias(fecha_mas_ventas,"Fecha","ventas")

    print("------ Fechas con mayor importe de ventas ------")
    get_mas_ocurrencias(fechas_mas_monto_vendido,"Fecha","pesos")

    sys.exit(0)

def main():
    archivo_ventas = sys.argv[1]

    recepcion_sucursal1, envio_sucursal1 = Pipe(False)
    recepcion_sucursal2, envio_sucursal2 = Pipe(False)

    p1 = multiprocessing.Process(target=sucursal1_handler, args=(recepcion_sucursal1,envio_sucursal1))
    p2 = multiprocessing.Process(target=sucursal2_handler, args=(recepcion_sucursal2,envio_sucursal2))
    p1.start()
    p2.start()

    registros_sucursal1, registros_sucursal2 = procesar_ventas(archivo_ventas)

    recepcion_sucursal1.close()
    envio_sucursal1.send(registros_sucursal1)
    envio_sucursal1.close()

    recepcion_sucursal2.close()
    envio_sucursal2.send(registros_sucursal2)
    envio_sucursal2.close()

    p1.join()
    p2.join()

if __name__=="__main__":
    main()

Overwriting concurrencia.py


In [ ]:
!python concurrencia.py "ventas.txt"


Sucursal N°1
El monto total de ventas de la sucursal 1 es: $103150
------ Producto/s mas vendido/s: ------
Producto: pan | 2 unidades
Producto: leche | 2 unidades
Producto: manteca | 2 unidades
------ Fechas con más ventas ------
Fecha: 01-01-2024 | 5 ventas
------ Fechas con mayor importe de ventas ------
Fecha: 10-10-2024 | 85500.0 pesos

Sucursal N°2
El monto total de ventas de la sucursal 2 es: $38800
------ Producto/s mas vendido/s: ------
Producto: fanta | 5 unidades
------ Fechas con más ventas ------
Fecha: 10-11-2024 | 3 ventas
Fecha: 29-12-2024 | 3 ventas
Fecha: 30-12-2024 | 3 ventas
------ Fechas con mayor importe de ventas ------
Fecha: 10-11-2024 | 5700 pesos
Fecha: 29-12-2024 | 5700 pesos
Fecha: 30-12-2024 | 5700 pesos
